In [1]:
import keras
import sys
import os
import cv2
import numpy as np

from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D,BatchNormalization
from keras import backend as K
from keras import regularizers
from keras.utils import plot_model

import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
def load_imgs():
    
    train_x=[]
    train_y=[]
    for category in os.listdir('./train'):
            print ('loading category {}...'.format(category))
            label = category[-1]
            for image in os.listdir('./train/' + category):
                f = ('./train/' + category + '/' + image)
                train_x.append(cv2.imread(f, 0))
                train_y.append(label)
                
    train_x=np.array(train_x,dtype=np.float32)
    train_x=train_x.reshape(train_x.shape[0],train_x.shape[1],train_x.shape[2],1)
    train_x/=255
    train_y=np.array(train_y,dtype=np.float32)
    
    val_x=[]
    val_y=[]
    for category in os.listdir('./valid'):
            label = category[-1]
            for image in os.listdir('./valid/' + category):
                f = ('./valid/' + category + '/' + image)
                val_x.append(cv2.imread(f,0))
                val_y.append(label)
                   
    val_x=np.array(train_x,dtype=np.float32)
    val_x=train_x.reshape(train_x.shape[0],train_x.shape[1],train_x.shape[2],1)
    val_x/=255
    val_y=np.array(train_y,dtype=np.float32)
    
    return train_x,train_y,val_x,val_y

In [3]:

train_x,train_y,val_x,val_y = load_imgs()

loading category class_8...
loading category class_9...
loading category class_2...
loading category class_0...
loading category class_4...
loading category class_3...
loading category class_1...
loading category class_5...
loading category class_6...
loading category class_7...


In [4]:
set(train_y)

{0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0}

In [5]:
train_x.shape

(50000, 28, 28, 1)

In [6]:
(x_train, y_train), (x_test, y_test) = (train_x,train_y),(val_x,val_y)

In [7]:
import tensorflow as tf
import numpy      as np
import os
import argparse
import keras

from keras.models               import Sequential, load_model
from keras.layers.normalization import BatchNormalization
from keras.optimizers           import Adam, Adadelta
from keras.utils                import np_utils
from keras                      import regularizers
from keras.preprocessing.image  import ImageDataGenerator
from keras.layers               import Dense, Dropout, Activation, Flatten
from keras.layers               import Convolution2D, Conv2D, AveragePooling2D
from keras.layers               import ZeroPadding2D, MaxPooling2D
from keras.callbacks            import ModelCheckpoint,EarlyStopping


In [8]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print (x_train.shape)


(60000, 28, 28)


In [10]:
batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28


# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print (x_train.shape)

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
model.summary()
filepath='save/Model.{epoch:02d}-{val_acc:.4f}.hdf5'
checkpoint1 = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint1]
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test), 
         callbacks=callbacks_list)
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


(60000, 28, 28)
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               1179776   
_____________________________________________________